In [10]:
# library
import os
import json
import base64
import requests
import pandas as pd
import tensorflow as tf

In [11]:
# check
from pprint import PrettyPrinter

pp = PrettyPrinter()
pp.pprint(requests.get("http://localhost:8080/v1/models/movie-reviews-sentiment-model").json())

{'model_version_status': [{'state': 'AVAILABLE',
                           'status': {'error_code': 'OK', 'error_message': ''},
                           'version': '1681103188'}]}


In [12]:
# path
DATA_PATH = 'data'

df = pd.read_csv(os.path.join(DATA_PATH, 'rotten_tomatoes.csv'))
text = df['text'][18]
text

"the movie's ripe , enrapturing beauty will tempt those willing to probe its inscrutable mysteries ."

In [13]:
def prepare_json(text):
    feature_spec = {
        'text': tf.train.Feature(bytes_list=tf.train.BytesList(value=[bytes(text, 'utf-8')]))
    }

    example = tf.train.Example(
        features=tf.train.Features(feature=feature_spec)
    ).SerializeToString()

    result = [{
        'examples': {
            'b64': base64.b64encode(example).decode()
        }
    }]

    return json.dumps({
        'signature_name': 'serving_default',
        'instances': result
    })

In [14]:
# function to predict
def make_predictions(text):
    json_ = prepare_json(text)
    endpoint = 'http://localhost:8080/v1/models/movie-reviews-sentiment-model:predict'

    response = requests.post(endpoint, data=json_)

    predict = response.json()['predictions'][0][0]
    return predict

In [15]:
# predict
predict = make_predictions(text)

print(f'{text}\n')
print(f'Movie reviews are positive sentiment ({predict})' if predict > 0.6 else f'Movie reviews are negative sentiment ({predict})')

the movie's ripe , enrapturing beauty will tempt those willing to probe its inscrutable mysteries .

Movie reviews are positive sentiment (0.947859704)
